In [73]:
import numpy as np
import optuna
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
import xgboost as xgb


## Reading in Data

In [74]:
x_train = pd.read_csv(
  "../data/processed/x_train_w_OHE.csv", index_col=0, dtype=str
)
x_test = pd.read_csv(
  "../data/processed/x_test_w_OHE.csv", index_col=0, dtype=str
)
y_train = pd.read_csv(
  "../data/processed/y_train.csv", index_col=0, dtype=float
).squeeze("columns").reset_index(drop=True)
y_test = pd.read_csv(
  "../data/processed/y_test.csv", index_col=0, dtype=float
).squeeze("columns").reset_index(drop=True)

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42)

In [75]:
def get_correct_types_x(df, numeric_cols):
    for col in ['deenergize_time', 'restoration_time']:
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
    for col in numeric_cols:
        df[col] = df[col].astype(float)
    return df
numeric_cols = [
    'hftd_tier', 'total_affected', 'residential_affected',
    'longitude', 'latitude', 'total_pop', 'median_age', 'median_income',
    'white_pct', 'tmin_d-5', 'tmax_d-5', 'wspd_d-5', 'tmin_d-4', 'tmax_d-4',
    'wspd_d-4', 'tmin_d-3', 'tmax_d-3', 'wspd_d-3', 'tmin_d-2', 'tmax_d-2',
    'wspd_d-2', 'tmin_d-1', 'tmax_d-1', 'wspd_d-1'
]

In [76]:
#Scale all numeric columns then add back in zip columns
zip_cols = x_train.columns[
    [re.search('zip_is', col) is not None for col in x_train.columns]
]
x_train = get_correct_types_x(x_train, numeric_cols)
x_valid = get_correct_types_x(x_valid, numeric_cols)
x_test = get_correct_types_x(x_test, numeric_cols)
rel_x_train = x_train[numeric_cols]
rel_x_valid = x_valid[numeric_cols]
rel_x_test = x_test[numeric_cols]

scaler = StandardScaler()
scaler.fit(rel_x_train)
scaled_x_train = scaler.transform(rel_x_train)
scaled_x_valid = scaler.transform(rel_x_valid)
scaled_x_test = scaler.transform(rel_x_test)

## Baseline Model

In [77]:
baseline_params = {'max_depth':6, 'eta':.3, 'objective':'reg:squarederror'}
num_round = 5
baseline_d_train = xgb.DMatrix(scaled_x_train, label = y_train)
xgb_model = xgb.train(baseline_params, baseline_d_train, num_round)
d_test = xgb.DMatrix(scaled_x_test, label = y_test)
baseline_preds = xgb_model.predict(d_test)
print("RMSE = ", np.sqrt(mean_squared_error(baseline_preds, y_test)))

RMSE =  1091.158592436061


## Optimizing Hyperparameters

In [78]:
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }

    # Fit the model
    optuna_model = xgb.XGBRegressor(**params)
    optuna_model.fit(scaled_x_train, y_train)

    # Make predictions
    y_valid_pred = optuna_model.predict(scaled_x_valid)

    # Evaluate predictions
    accuracy = np.sqrt(mean_squared_error(y_valid_pred, y_valid))
    return accuracy

In [79]:
study = optuna.create_study(direction='minimize')

[I 2022-11-26 18:16:34,708] A new study created in memory with name: no-name-5a4bfb0b-682f-4a20-a3f4-dc011100d362


In [80]:
study.optimize(objective, n_trials=100)
warnings.filterwarnings('ignore')

print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  RMSE: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2022-11-26 18:16:34,974] Trial 0 finished with value: 725.9818387602426 and parameters: {'max_depth': 7, 'learning_rate': 0.03248464712791111, 'n_estimators': 304, 'min_child_weight': 5, 'gamma': 1.4182761007612066e-07, 'subsample': 0.9704910651418149, 'colsample_bytree': 0.13124642958552749, 'reg_alpha': 0.001900487203428946, 'reg_lambda': 0.0027834283322071976}. Best is trial 0 with value: 725.9818387602426.
[I 2022-11-26 18:16:35,143] Trial 1 finished with value: 903.5100122820469 and parameters: {'max_depth': 6, 'learning_rate': 0.0772691964328607, 'n_estimators': 490, 'min_child_weight': 4, 'gamma': 0.02799117593178613, 'subsample': 0.08644301450628056, 'colsample_bytree': 0.05900231955498274, 'reg_alpha': 0.004699638079366631, 'reg_lambda': 0.00037186051867927494}. Best is trial 0 with value: 725.9818387602426.
[I 2022-11-26 18:16:35,199] Trial 2 finished with value: 1229.646620942435 and parameters: {'max_depth': 4, 'learning_rate': 0.014921958943203818, 'n_estimators': 328, 

Number of finished trials: 100
Best trial:
  RMSE: 703.2232083132906
  Params: 
    max_depth: 7
    learning_rate: 0.05486946254183157
    n_estimators: 212
    min_child_weight: 10
    gamma: 2.163425403087385e-08
    subsample: 0.7306303686779471
    colsample_bytree: 0.8826965695361724
    reg_alpha: 0.0014225927303242439
    reg_lambda: 8.07358757093701e-07


## Fit model with best parameters and predict

In [81]:
def calc_test_r2(pred_vals, true_vals, baseline_rmse):
    sse = mean_squared_error(pred_vals, true_vals) * len(true_vals)
    sst = (baseline_rmse ** 2) * len(true_vals)
    return 1 - sse / sst, np.sqrt(sse / len(true_vals))

In [82]:
best_params = trial.params
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(scaled_x_train, y_train)
final_preds = best_model.predict(scaled_x_test)
# print("RMSE = ", np.sqrt(mean_squared_error(final_preds, y_test)))
baseline_rmse = np.sqrt(((y_test - y_test.mean()) ** 2).mean())
test_r2, rmse = calc_test_r2(final_preds, y_test, baseline_rmse)
print('Test R-Squared:', test_r2)
print('RMSE:', rmse)

Test R-Squared: 0.7682088317177024
RMSE: 760.6811004762943
